## Final Exam Machine Learning

Davide Tateo; 167275

Francesca Salute; 167284

Nicole Favero; 167340 

Tomás Gonçalves; 167288

In [1]:
!git clone https://github.com/frasalute/ML_FinalExam.git

Cloning into 'ML_FinalExam'...
remote: Enumerating objects: 22331, done.
remote: Counting objects: 100% (8143/8143), done.
remote: Compressing objects: 100% (8128/8128), done.
remote: Total 22331 (delta 22), reused 8132 (delta 14), pack-reused 14188
Receiving objects: 100% (22331/22331), 401.90 MiB | 35.18 MiB/s, done.
Resolving deltas: 100% (33/33), done.
Updating files: 100% (22257/22257), done.


In [2]:
import os
print(f"Current working directory: {os.getcwd()}")
print(f"Contents of the current directory: {os.listdir('.')}")

Current working directory: /kaggle/working
Contents of the current directory: ['__notebook__.ipynb', 'ML_FinalExam']


In [3]:
base_path = '/kaggle/working/ML_FinalExam/images'
print("Base path is:", base_path)

Base path is: /kaggle/working/ML_FinalExam/images


In [4]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

2024-05-09 20:22:32.721789: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-09 20:22:32.721886: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-09 20:22:32.858220: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
#importing required API and modules
import warnings
import numpy as np
import pandas as pd
from tensorflow.keras.utils import img_to_array
from keras.preprocessing.image import array_to_img
from tensorflow.keras.utils import load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
def load_images_to_df(base_path):
    data = {'photo_id': [], 'image': [], 'image_array' :[], 'malignant': []}
    categories = {'Benign': 0, 'Malignant': 1}
    
    for subset in ['train', 'test']:
        for category in ['Benign', 'Malignant']:
            folder_path = os.path.join(base_path, subset, category)
            for filename in os.listdir(folder_path):
                if filename.endswith('.jpg'):
                    file_path = os.path.join(folder_path, filename)
                    # Upload Images
                    image = load_img(file_path)
                    image_array = img_to_array(image)
                    # Add the data
                    data['photo_id'].append(filename)
                    data['image'].append(image)
                    data['image_array'].append(image_array)
                    data['malignant'].append(categories[category])
    
    # Create DataFrame
    df = pd.DataFrame(data)
    return df


df = load_images_to_df(base_path)


In [7]:
print(df.head())
row_count = len(df)
print("Number of rows in the DataFrame:", row_count)

   photo_id                                              image  \
0  5366.jpg  <PIL.JpegImagePlugin.JpegImageFile image mode=...   
1  4347.jpg  <PIL.JpegImagePlugin.JpegImageFile image mode=...   
2    90.jpg  <PIL.JpegImagePlugin.JpegImageFile image mode=...   
3  5524.jpg  <PIL.JpegImagePlugin.JpegImageFile image mode=...   
4  5006.jpg  <PIL.JpegImagePlugin.JpegImageFile image mode=...   

                                         image_array  malignant  
0  [[[147.0, 128.0, 132.0], [150.0, 131.0, 135.0]...          0  
1  [[[189.0, 120.0, 139.0], [188.0, 119.0, 138.0]...          0  
2  [[[242.0, 167.0, 188.0], [241.0, 166.0, 187.0]...          0  
3  [[[196.0, 154.0, 166.0], [195.0, 153.0, 165.0]...          0  
4  [[[181.0, 109.0, 133.0], [179.0, 107.0, 131.0]...          0  
Number of rows in the DataFrame: 13879


Code to make augmentation of pics

In [8]:
'''original_dataset_dir = './images/undetec_to_augm'
train_augm_undetected_dir = './images/train/Undetected'
test_augm_undetected_dir ='./images/test/Undetected'

# Create a data generator for augmentation
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range = [0.8, 1.3],
    fill_mode='nearest'
)

# List the files in the original dataset directory
file_list = os.listdir(original_dataset_dir)

# Ensure the target directories exist
os.makedirs(train_augm_undetected_dir, exist_ok=True)
os.makedirs(test_augm_undetected_dir, exist_ok=True)

# Desired number of images after augmentation
target_count_1 = 6000
target_count_2 = 1000
"""= target_count_2 = target_count_3 = target_count_4 = target_count_5 = target_count_6 = target_count_7"""

# Number of images in the original dataset
original_count = len(file_list)


# Number of times to repeat each image to reach the target count
repeats_1 = min(target_count_1 // original_count + 1, len(file_list))
repeats_2 = min(target_count_2 // original_count + 1, len(file_list))
"""repeats_3 = min(target_count_3 // original_count + 1, len(file_list))
repeats_4 = min(target_count_4 // original_count + 1, len(file_list))
repeats_5 = min(target_count_5 // original_count + 1, len(file_list))
repeats_6 = min(target_count_6 // original_count + 1, len(file_list))
repeats_7 = min(target_count_7 // original_count + 1, len(file_list))"""

# Augment and save the images for the 6000 train images
for file in file_list[:original_count]:
    img_path = os.path.join(original_dataset_dir, file)
    img = load_img(img_path)
    img = img_to_array(img)
    img = img.reshape((1,) + img.shape)

    i = 0
    for batch in datagen.flow(img, batch_size=1, save_to_dir = train_augm_undetected_dir, save_prefix='augm', save_format='jpeg'): 
        #if in the previous line i save them as jpg, it anyway augments only the jpeg ones
        i += 1
        if i >= repeats_1:
            break  # break the loop after reaching the desired number of augmented images



# Augment and save the images for the 1000 test images
for file in file_list[:original_count]:
    img_path = os.path.join(original_dataset_dir, file)
    img = load_img(img_path)
    img = img_to_array(img)
    img = img.reshape((1,) + img.shape)

    i = 0
    for batch in datagen.flow(img, batch_size=1, save_to_dir = test_augm_undetected_dir, save_prefix='aug', save_format='jpeg'):
        i += 1
        if i >= repeats_2:
            break  # break the loop after reaching the desired number of augmented images '''

'original_dataset_dir = \'./images/undetec_to_augm\'\ntrain_augm_undetected_dir = \'./images/train/Undetected\'\ntest_augm_undetected_dir =\'./images/test/Undetected\'\n\n# Create a data generator for augmentation\ndatagen = ImageDataGenerator(\n    rotation_range=30,\n    width_shift_range=0.2,\n    height_shift_range=0.2,\n    shear_range=0.2,\n    zoom_range=0.2,\n    horizontal_flip=True,\n    brightness_range = [0.8, 1.3],\n    fill_mode=\'nearest\'\n)\n\n# List the files in the original dataset directory\nfile_list = os.listdir(original_dataset_dir)\n\n# Ensure the target directories exist\nos.makedirs(train_augm_undetected_dir, exist_ok=True)\nos.makedirs(test_augm_undetected_dir, exist_ok=True)\n\n# Desired number of images after augmentation\ntarget_count_1 = 6000\ntarget_count_2 = 1000\n"""= target_count_2 = target_count_3 = target_count_4 = target_count_5 = target_count_6 = target_count_7"""\n\n# Number of images in the original dataset\noriginal_count = len(file_list)\n\n\n